<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr2/blob/main/Yurkov%20Vladimir/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparations

Импортируем модули

In [1]:
import pandas as pd
import numpy as np
import gdown
import matplotlib.pyplot as plt
import re
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

Загружаем данные и преобразуем их в pandas dataframe

In [2]:
url = 'https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx'
file_name = 'data.xlsx'
gdown.download(url, file_name, quiet=False)
df_backup = pd.read_excel(file_name)
df_backup.head()

Downloading...
From: https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx
To: /content/data.xlsx
100%|██████████| 32.4M/32.4M [00:03<00:00, 10.3MB/s]


,ДокументПрохождениеМедосмотра,Клиент,КлиентДатаРождения,КлиентПол,КлиентПолПредставление,ДатаЗавершенияМедосмотра,ТипМедосмотра,ТипМедосмотраПредставление,ПсихОсвидетельствование,Контрагент,...,Оториноларингология3_Аудиометрия1_Значение,Оториноларингология3_Аудиометрия1_ЗначениеПредставление,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление,Стоматология2_ВП2_Значение,Стоматология2_ВП2_ЗначениеПредставление,Терапия2_ЗаключениеФОГК_Значение,Терапия2_ЗаключениеФОГК_ЗначениеПредставление,Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение,Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
0,7f41ca7e-7dac-11ec-80fc-0cc47aab8067,c91e3407-b81f-11e9-80cf-0cc47aab8067,1976-10-24,Женский,Женский,2022-01-25,Периодический,Периодический,False,23c173a7-30d9-11e7-80d8-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2ee1f9ac-83dd-11ec-80fc-0cc47aab8067,1a1a5fea-8b7f-11eb-80e8-0cc47aab8067,1987-11-18,Мужской,Мужской,2022-02-02,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9cbb191e-8e24-11ec-80fe-0cc47aab8067,7fe324e0-fa51-11eb-80f6-0cc47aab8067,1985-09-08,Мужской,Мужской,2022-02-15,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b8a33910-8eec-11ec-80fe-0cc47aab8067,98f9d761-8d65-11ec-80fe-0cc47aab8067,1988-04-25,Мужской,Мужской,2022-02-16,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0bb0d181-938c-11ec-80fe-0cc47aab8067,009aac7d-7bcf-11eb-80e8-0cc47aab8067,1984-04-17,Мужской,Мужской,2022-02-22,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Getting started with a dataframe

Делаем копию датафрейма

In [3]:
df = df_backup.copy()

Блок с функциями.

* `clean_clear` - функция, удаляющая дубликаты строк/столбцов и полностью пустые
строки/столбцы.
* `regex_to_nan` - функция, находящая значения по регулярному выражению и заменяющая его на `np.nan`.
* `fills_count` - функция, возвращающая заполненность датафрейма в процентах.

In [4]:
def clean_clear(df, pflag=False):
    # Сохраняем размерность датафрейма
    shape_sum = df.shape

    # Удаляем дубликаты строк
    df = df.drop_duplicates()
    # Удаляем дубликаты столбцов
    df = df.T.drop_duplicates().T

    # Удаляем полностью пустые столбцы и строки
    df = df.dropna(axis=0, how='all')
    df = df.dropna(axis=1, how='all')

    # Выводим результаты
    if pflag:
        print(f"Форма датасета до операции: {shape_sum}")
        print(f"Форма датасета после операции: {df.shape}")

        print(f"Количество строк после удаления пустых и повторяющихся: {df.shape[0]}")
        print(f"Количество столбцов после удаления пустых и повторяющихся: {df.shape[1]}")
        print(f"\n*Удалено {shape_sum[0] - df.shape[0]} строк и {shape_sum[1] - df.shape[1]} столбцов.")
    df = df.convert_dtypes()
    return df

# Регулярное выражения для GUID
regex_GUID = "^[{]?[0-9a-fA-F]{8}" + "-([0-9a-fA-F]{4}-)" + "{3}[0-9a-fA-F]{12}[}]?$"

# Поиск и удаление GUID столбцов
def regex_to_nan(df, to_replace=regex_GUID, value=np.nan, pflag=False):
    df = df.replace(to_replace=to_replace, value=value, regex=True)
    if pflag:
        print(f"Колонки для удаления:\n{df.columns[df.isna().all()].tolist()}")
    return df

# Подсчет заполненности таблицы
def fills_count(df):
    return f"Заполненность: ~{round((1 - (df.isna().sum().sum() / df.size))*100, 2)}%"

Анализ данных.
* форма
* количество значений
* основная информация о датафрейме
* определение и подсчет типов данных до и после попытки автоматического преобразования типов `df.convert_dtypes()`
* процент заполненности данными

In [5]:
print()
print("Форма: ", df.shape)
print("Количество значений: ", df.size)
print()
display(df.info(max_cols=0))
# pd.set_option('display.max_rows', None) # (default=20) для сброса pd.reset_option('display.max_rows') | для колонок max_columns
# display(df.dtypes)
# pd.reset_option('display.max_rows')
print("\nТипы данных в фрейме:")
display(df.dtypes.value_counts())
print("\nПреобразование типов данных.")
df = df.convert_dtypes()
print("\nТипы данных после преобразования:")
display(df.dtypes.value_counts())
print()
print(fills_count(df))


Форма:  (1982, 4587)
Количество значений:  9091434

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982 entries, 0 to 1981
Columns: 4587 entries, ДокументПрохождениеМедосмотра to Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
dtypes: bool(1), datetime64[ns](43), float64(1253), object(3290)
memory usage: 69.3+ MB


None


Типы данных в фрейме:


object            3290
float64           1253
datetime64[ns]      43
bool                 1
dtype: int64


Преобразование типов данных.

Типы данных после преобразования:


string            3206
Int64             1228
boolean             85
datetime64[ns]      43
Float64             25
dtype: int64


Заполненность: ~10.83%


Поиск колонок с датами.
* нахождение по обнаруженному типу `datetime64[ns]`
* нахождение колонок, содержащих в названии слово `Дата`

Сопоставление полученных результатов между собой.
Получилось, что данные по слову `Дата` имеют действительно даты, только не определились и полностью включают в себя найденные стобцы по автоматически определенному типу данных - `datetime64[ns]`

In [6]:
datatype_list = df.columns[df.dtypes == 'datetime64[ns]'].values.tolist()
print(f"Количество столбцов с типом 'datetime64[ns]': {len(datatype_list)}")
dataword_list = df.filter(regex=re.compile("дата", re.IGNORECASE), axis=1).columns.values.tolist()
print(f"Количество столбцов со словом 'Дата': {len(dataword_list)}")
print()
if len(datatype_list) > len(dataword_list):
    data_diff = set(datatype_list) - set(dataword_list)
    if set(dataword_list).issubset(datatype_list):
        print("Список dataword_list входит в datatype_list\n")
elif len(datatype_list) < len(dataword_list):
    data_diff =  set(dataword_list) - set(datatype_list)
    if set(datatype_list).issubset(dataword_list):
        print("Список datatype_list входит в dataword_list\n")
else: print("Списки равны\n")

# Для вывода списка разницы колонок убрать комментарий с строки
# print(*data_diff, sep="\n")

Количество столбцов с типом 'datetime64[ns]': 43
Количество столбцов со словом 'Дата': 111

Список datatype_list входит в dataword_list



Ищем колонки с Ф.И.О. врачей и названием медицинского учреждения

In [7]:
datadoctor_list = df.filter(regex=re.compile("врач", re.IGNORECASE), axis=1).columns.values.tolist()
print(f"Колчество колонок с Ф.И.О. врачей: {len(datadoctor_list)}")
datasalon_list = df.filter(regex=re.compile("салон", re.IGNORECASE), axis=1).columns.values.tolist()
print(f"Колчество колонок с названием мед. учреждения: {len(datasalon_list)}")
dataspec_list = df.filter(regex=re.compile("специализация", re.IGNORECASE), axis=1).columns.values.tolist()
print(f"Колчество колонок с названием специализации: {len(datasalon_list)}")

Колчество колонок с Ф.И.О. врачей: 66
Колчество колонок с названием мед. учреждения: 86
Колчество колонок с названием специализации: 86


Анализ и проверка целевой колонки `ЗаключениеМК` на предмет "мусорных" данных и пропусков.

In [8]:
display(df['ЗаключениеМК'].unique().tolist())

['Годен',
 'ГоденСКоррекциейЗрения',
 'ГоденБезРаботНаВысотах',
 'НуждаетсяВДообследованииИЛечении',
 'ВременноНегоден',
 'ОграниченноГоден',
 <NA>,
 'машинист бульдозера',
 'Газоспасатель',
 ' Газоспасатель']

Создание маски с соответствующими критериям данными для целевой колонки.

In [9]:
mask_list = ['Годен', 'ГоденСКоррекциейЗрения', 'ГоденБезРаботНаВысотах',
       'НуждаетсяВДообследованииИЛечении', 'ВременноНегоден',
       'ОграниченноГоден']

Чистка от мусорных, дублирующих, не имеющих ценности и пустых данных.

Сначала производится чистка по маске из целевой колонки `ЗаключениеМК`, затем удаляются данные с датами, затем данные с GUID, дублирующие и пустые.

Сравнивается форма датафрейма до и после чистки.

In [10]:
print(f"Форма датафрейма до: {df.shape}")
try:
    print("Чистка от некорректных данных в целевой колонке")
    df = df[df['ЗаключениеМК'].isin(mask_list)]
    print(df.shape)

    print("Чистка от колонок с датами")
    df = df.drop(dataword_list, axis='columns')
    print(df.shape)

    print("Чистка от колонок с Ф.И.О. врачей")
    df = df.drop(datadoctor_list, axis='columns')
    print(df.shape)

    print("Чистка от колонок с названием мед. учреждений")
    df = df.drop(datasalon_list, axis='columns')
    print(df.shape)

    print("Чистка от колонок с названием специализации врача")
    df = df.drop(dataspec_list, axis='columns')
    print(df.shape)

    print("Чистка от колонок с GUID, полностью пустых колонок и строк, дубликатов колонок и строк...")
    df = regex_to_nan(df)
    df = clean_clear(df)
except Exception as e:
    print(e)
print(f"Форма датафрейма после: {df.shape}")

Форма датафрейма до: (1982, 4587)
Чистка от некорректных данных в целевой колонке
(1571, 4587)
Чистка от колонок с датами
(1571, 4476)
Чистка от колонок с Ф.И.О. врачей
(1571, 4410)
Чистка от колонок с названием мед. учреждений
(1571, 4324)
Чистка от колонок с названием специализации врача
(1571, 4238)
Чистка от колонок с GUID, полностью пустых колонок и строк, дубликатов колонок и строк...
Форма датафрейма после: (1446, 1473)


#Categorization, OHE, Tokenization, BoW, etc...

In [40]:
# hardcode
lis = df.filter(regex=re.compile('ЛабораторныеИсследования1_Кал', re.IGNORECASE), axis=1).columns.values.tolist()
d = {}
for i in lis:
    d[i] = (df[i].unique())
print(d)

{'ЛабораторныеИсследования1_Кал_Значение': <StringArray>
['не получен', 'получен', <NA>]
Length: 3, dtype: string}


Проверим типы данных в датафрейме

In [41]:
display(df.dtypes.value_counts())

string     1282
object      151
boolean      40
dtype: int64

###functions

Функция для просмотра уникальных данных в фрейме для каждого столбца, возвращающая словарь

In [42]:
def display_nunique_cond(frame):
    dict_to_return = {}
    for i in frame.columns:
        dict_to_return[i] = frame[i].unique()
    print(f"Количество: {len(dict_to_return)}")
    return dict_to_return

###x_0, x_1, x_2, y
`x_0` - данные для обучения, состоящие из df_object и df_bool, привиденные в float и нормализованные

`x_1` - данные с значениями МКБ и Вредными факторами, приведенные к категорям

`x_2` - Остальные текстовые данные с токенизацией

####`df` separationby dtype -> `df_object`, `df_bool`, `df_string`

Разделим наш датафрейм на датафреймы по типам данных

In [43]:
column_object_list = df.columns[df.dtypes == 'object'].values.tolist()
column_bool_list = df.columns[df.dtypes == 'boolean'].values.tolist()
column_string_list = df.columns[df.dtypes == 'string'].values.tolist()

df_object = df[column_object_list]
df_bool = df[column_bool_list]
df_string = df[column_string_list]

print("df_object - датафрейм с типом данных object")
print(fills_count(df_object))
display(df_object.head())
print()

print("df_bool - датафрейм с типом данных boolean")
print(fills_count(df_bool))
display(df_bool.head())
print()

print("df_string - датафрейм с типом данных string")
print(fills_count(df_string))
display(df_string.head())
print()

df_object - датафрейм с типом данных object
Заполненность: ~18.23%


,ПсихиатрияНаркология1_Профессиональное1,ПсихиатрияНаркология1_Чистая_Значение,ПсихиатрияНаркология1_СоСледамиВВИнъекций_Значение,ПсихиатрияНаркология1_Контактен_Значение,ПсихиатрияНаркология1_НеКонтактен_Значение,Офтальмология1_Профессиональное1,Офтальмология1_ДискиЗрительныхНервовБледноРозовые_Значение,Оториноларингология1_Профессиональное1,Оториноларингология1_ЗаДужками_Значение,Оториноларингология1_Отсутствуют_Значение,...,ТерапевтНеИсп1_АртериальноеДавление_Значение,ТерапевтНеИсп1_П_Значение,ТерапевтНеИсп1_Рост_Значение,ТерапевтНеИсп1_Вес_Значение,ТерапевтНеИсп1_ТемператураТелаТерапевт_Значение,ТерапевтНеИсп1_ИМТ_Значение,ТерапевтНеИсп1_АртериальноеДавление21_Значение,ТерапевтНеИсп1_Сатурация_Значение,Терапия3_ОбщийХолестерин_Значение,Гинекология2_Выкидышей_Значение
0,0,1,0,0,0,0,0,0,1,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,0,1,0,1,0,0,1,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,0,<NA>,<NA>,0,0,0,0,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,0,1,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,0,1,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>



df_bool - датафрейм с типом данных boolean
Заполненность: ~4.32%


,ПсихОсвидетельствование,Оториноларингология2_Профессиональное1,Дерматовенерология1_Профессиональное2,Дерматовенерология2_Профессиональное1,Дерматовенерология2_Профессиональное2,Хирургия1_Профессиональное2,Офтальмология2_Профессиональное1,Офтальмология2_Профессиональное2,Офтальмология2_ДискиЗрительныхНервовБледноРозовые_Значение,Офтальмология2_СетчаткаПрилежит_Значение,...,Оториноларингология2_ЗаДужками_Значение,Оториноларингология2_Отсутствуют_Значение,ТерапевтНеИсп1_ЭКГПроведено_Значение,Эндоскопия1_Профессиональное1,ТерапевтНеИсп1_Профессиональное2,ХирургНеИсп1_Профессиональное2,Психиатрия1_Профессиональное2,Рентгенология1_Профессиональное2,Психиатрия3_Профессиональное1,Оториноларингология3_Профессиональное1
0,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>



df_string - датафрейм с типом данных string
Заполненность: ~16.01%


,КлиентПол,ТипМедосмотра,Цех,Профессия,ЗаключениеМК,ЗаключениеМКПредставление,Заключение,ДиспансернаяГруппа,ВредныеФакторы,ЛабораторныеИсследования1_ОАК_Значение,...,Оториноларингология3_Диагноз_Значение,Оториноларингология3_ВидимыеСлизистые_ЗначениеПредставление,Оториноларингология3_СВысыпаниями_ЗначениеПредставление,Оториноларингология3_НебныеМиндалины1_ЗначениеПредставление,Оториноларингология3_НебныеМиндалины2_ЗначениеПредставление,Оториноларингология3_ВестибулярныйАнализатор1_ЗначениеПредставление,Оториноларингология1_ЖалобыДополнение_Значение,Стоматология2_ВП2_Значение,Терапия2_ЗаключениеФОГК_Значение,Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение
0,Женский,Периодический,<NA>,Ведущий специалист по организации обуения,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,I,11.4,получен,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,Мужской,Периодический,<NA>,Ведущий специалист,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,IIIа,4.2.5,получен,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,Мужской,Периодический,<NA>,Главный менеджер,ГоденСКоррекциейЗрения,Годен к работе в указанных условиях (с коррекц...,Годен к работе в указанных условиях (с коррекц...,II,4.2.5,получен,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,Мужской,Периодический,Отдел техники и технологии строительства скважин,Главный специалист,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,I,11.1,получен,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,Мужской,Периодический,Отдел технологий текущего и капитального ремон...,Старший специалист,ГоденСКоррекциейЗрения,Годен к работе в указанных условиях (с коррекц...,Годен к работе в указанных условиях (с коррекц...,II,11.1,получен,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


####**x_0**

**1.** df_object dtype `object`->`float`

Нам необходимо привести датафреймы к цифровому виду.
Посмотрим, что за данные `object`

In [73]:
display(display_nunique_cond(df_object))

Количество: 151


{'ПсихиатрияНаркология1_Профессиональное1': array([ 0., -1.]),
 'ПсихиатрияНаркология1_Чистая_Значение': array([ 1., -1.]),
 'ПсихиатрияНаркология1_СоСледамиВВИнъекций_Значение': array([ 0., -1.]),
 'ПсихиатрияНаркология1_Контактен_Значение': array([ 0.,  1., -1.]),
 'ПсихиатрияНаркология1_НеКонтактен_Значение': array([ 0., -1.]),
 'Офтальмология1_Профессиональное1': array([ 0., -1.]),
 'Офтальмология1_ДискиЗрительныхНервовБледноРозовые_Значение': array([ 0.,  1., -1.]),
 'Оториноларингология1_Профессиональное1': array([ 0., -1.]),
 'Оториноларингология1_ЗаДужками_Значение': array([ 1., -1.,  0.]),
 'Оториноларингология1_Отсутствуют_Значение': array([ 0., -1.,  1.]),
 'Оториноларингология1_Увеличены_Значение': array([ 0., -1.,  1.]),
 'Оториноларингология1_УвеличеныСПробками_Значение': array([ 0., -1.]),
 'ХирургНеИсп1_Профессиональное1': array([ 0., -1.]),
 'ХирургНеИсп1_Мягкий_Значение': array([ 0., -1.]),
 'Гинекология1_Профессиональное1': array([ 0., -1.]),
 'Гинекология1_Регулярны

Данные похожи на числа, попробуем преобразовать в тип float

In [74]:
try:
    display(df_object.astype(float))
except Exception as e:
    print(e)

,ПсихиатрияНаркология1_Профессиональное1,ПсихиатрияНаркология1_Чистая_Значение,ПсихиатрияНаркология1_СоСледамиВВИнъекций_Значение,ПсихиатрияНаркология1_Контактен_Значение,ПсихиатрияНаркология1_НеКонтактен_Значение,Офтальмология1_Профессиональное1,Офтальмология1_ДискиЗрительныхНервовБледноРозовые_Значение,Оториноларингология1_Профессиональное1,Оториноларингология1_ЗаДужками_Значение,Оториноларингология1_Отсутствуют_Значение,...,ТерапевтНеИсп1_АртериальноеДавление_Значение,ТерапевтНеИсп1_П_Значение,ТерапевтНеИсп1_Рост_Значение,ТерапевтНеИсп1_Вес_Значение,ТерапевтНеИсп1_ТемператураТелаТерапевт_Значение,ТерапевтНеИсп1_ИМТ_Значение,ТерапевтНеИсп1_АртериальноеДавление21_Значение,ТерапевтНеИсп1_Сатурация_Значение,Терапия3_ОбщийХолестерин_Значение,Гинекология2_Выкидышей_Значение
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1961,0.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1962,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1972,0.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1973,0.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


Есть ячейки без значения, проверим есть ли отрицательные числа

In [75]:
column_object_list = df_object.columns.values.tolist()
filter_ = (df_object[column_object_list] < 0).any(axis=1)
display(df_object[filter_])

,ПсихиатрияНаркология1_Профессиональное1,ПсихиатрияНаркология1_Чистая_Значение,ПсихиатрияНаркология1_СоСледамиВВИнъекций_Значение,ПсихиатрияНаркология1_Контактен_Значение,ПсихиатрияНаркология1_НеКонтактен_Значение,Офтальмология1_Профессиональное1,Офтальмология1_ДискиЗрительныхНервовБледноРозовые_Значение,Оториноларингология1_Профессиональное1,Оториноларингология1_ЗаДужками_Значение,Оториноларингология1_Отсутствуют_Значение,...,ТерапевтНеИсп1_АртериальноеДавление_Значение,ТерапевтНеИсп1_П_Значение,ТерапевтНеИсп1_Рост_Значение,ТерапевтНеИсп1_Вес_Значение,ТерапевтНеИсп1_ТемператураТелаТерапевт_Значение,ТерапевтНеИсп1_ИМТ_Значение,ТерапевтНеИсп1_АртериальноеДавление21_Значение,ТерапевтНеИсп1_Сатурация_Значение,Терапия3_ОбщийХолестерин_Значение,Гинекология2_Выкидышей_Значение
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1961,0.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1962,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1972,0.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1973,0.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


Отрицательные числа отсутствуют, заменим пустые значения на `-1` и сохраним как тип `float`

In [76]:
df_object = df_object.fillna(-1)
df_object = df_object.astype('float')
display(df_object.head())
display(df_object.dtypes.value_counts())

,ПсихиатрияНаркология1_Профессиональное1,ПсихиатрияНаркология1_Чистая_Значение,ПсихиатрияНаркология1_СоСледамиВВИнъекций_Значение,ПсихиатрияНаркология1_Контактен_Значение,ПсихиатрияНаркология1_НеКонтактен_Значение,Офтальмология1_Профессиональное1,Офтальмология1_ДискиЗрительныхНервовБледноРозовые_Значение,Оториноларингология1_Профессиональное1,Оториноларингология1_ЗаДужками_Значение,Оториноларингология1_Отсутствуют_Значение,...,ТерапевтНеИсп1_АртериальноеДавление_Значение,ТерапевтНеИсп1_П_Значение,ТерапевтНеИсп1_Рост_Значение,ТерапевтНеИсп1_Вес_Значение,ТерапевтНеИсп1_ТемператураТелаТерапевт_Значение,ТерапевтНеИсп1_ИМТ_Значение,ТерапевтНеИсп1_АртериальноеДавление21_Значение,ТерапевтНеИсп1_Сатурация_Значение,Терапия3_ОбщийХолестерин_Значение,Гинекология2_Выкидышей_Значение
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


float64    151
dtype: int64

**2.** df_bool dtype `boolean`->`float`

Посмотрим на данные в `df_bool`

In [77]:
# df_bool
display(display_nunique_cond(df_bool))

Количество: 40


{'ПсихОсвидетельствование': array([0., 1.]),
 'Оториноларингология2_Профессиональное1': array([-1.,  0.]),
 'Дерматовенерология1_Профессиональное2': array([-1.,  0.]),
 'Дерматовенерология2_Профессиональное1': array([-1.,  0.]),
 'Дерматовенерология2_Профессиональное2': array([-1.,  0.]),
 'Хирургия1_Профессиональное2': array([-1.,  0.]),
 'Офтальмология2_Профессиональное1': array([-1.,  0.]),
 'Офтальмология2_Профессиональное2': array([-1.,  0.]),
 'Офтальмология2_ДискиЗрительныхНервовБледноРозовые_Значение': array([-1.,  0.,  1.]),
 'Офтальмология2_СетчаткаПрилежит_Значение': array([-1.,  0.,  1.]),
 'Неврология2_Профессиональное2': array([-1.,  0.]),
 'ПсихиатрияНаркология1_Профессиональное2': array([-1.,  0.]),
 'УЗИДиагностика2_Профессиональное2': array([-1.,  0.]),
 'Оториноларингология1_Профессиональное2': array([-1.,  0.]),
 'Оториноларингология2_Профессиональное2': array([-1.,  0.]),
 'Терапия2_Профессиональное2': array([-1.,  0.]),
 'Оториноларингология1_Профессиональное3': a

Представим `df_bool` как данные типа `object`.
Заменим значения `<NA>` на `-1`, `False` на `0`, а `True` на `1`.
Сохраним `df_bool` как `float`

In [78]:
print("df_bool dtype(boolean -> object)")
df_bool = df_bool.astype('object')
print("df_bool Замена <NA> -> -1, False -> 0, True -> 1...")
df_bool = df_bool.fillna(-1).replace(False, 0).replace(True, 1)
df_bool = df_bool.astype('float')
display(df_bool.head())
display(df_bool.dtypes.value_counts())

df_bool dtype(boolean -> object)
df_bool Замена <NA> -> -1, False -> 0, True -> 1...


,ПсихОсвидетельствование,Оториноларингология2_Профессиональное1,Дерматовенерология1_Профессиональное2,Дерматовенерология2_Профессиональное1,Дерматовенерология2_Профессиональное2,Хирургия1_Профессиональное2,Офтальмология2_Профессиональное1,Офтальмология2_Профессиональное2,Офтальмология2_ДискиЗрительныхНервовБледноРозовые_Значение,Офтальмология2_СетчаткаПрилежит_Значение,...,Оториноларингология2_ЗаДужками_Значение,Оториноларингология2_Отсутствуют_Значение,ТерапевтНеИсп1_ЭКГПроведено_Значение,Эндоскопия1_Профессиональное1,ТерапевтНеИсп1_Профессиональное2,ХирургНеИсп1_Профессиональное2,Психиатрия1_Профессиональное2,Рентгенология1_Профессиональное2,Психиатрия3_Профессиональное1,Оториноларингология3_Профессиональное1
0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


float64    40
dtype: int64

**3.** `df_object` + `df_bool`

In [79]:
print(f"df_object.shape: {df_object.shape}")
print(f"df_bool.shape: {df_bool.shape}")
print("Конкатенация df_object и df_bool и запись фрейма в df_float")
df_0 = pd.concat([df_object, df_bool], axis='columns')
print(f"df_0.shape: {df_0.shape}")
print(df_0.dtypes.value_counts())

df_object.shape: (1446, 151)
df_bool.shape: (1446, 40)
Конкатенация df_object и df_bool и запись фрейма в df_float
df_0.shape: (1446, 191)
float64    191
dtype: int64


**4.** Normalization

In [80]:
for c in df_0.columns:
    if df_0[c].max() > 1:
        df_0[c] = df_0[c].astype('float32') / df_0[c].max()

In [81]:
df_0.describe()
x_1 = df_0.copy()

,ПсихиатрияНаркология1_Профессиональное1,ПсихиатрияНаркология1_Чистая_Значение,ПсихиатрияНаркология1_СоСледамиВВИнъекций_Значение,ПсихиатрияНаркология1_Контактен_Значение,ПсихиатрияНаркология1_НеКонтактен_Значение,Офтальмология1_Профессиональное1,Офтальмология1_ДискиЗрительныхНервовБледноРозовые_Значение,Оториноларингология1_Профессиональное1,Оториноларингология1_ЗаДужками_Значение,Оториноларингология1_Отсутствуют_Значение,...,Оториноларингология2_ЗаДужками_Значение,Оториноларингология2_Отсутствуют_Значение,ТерапевтНеИсп1_ЭКГПроведено_Значение,Эндоскопия1_Профессиональное1,ТерапевтНеИсп1_Профессиональное2,ХирургНеИсп1_Профессиональное2,Психиатрия1_Профессиональное2,Рентгенология1_Профессиональное2,Психиатрия3_Профессиональное1,Оториноларингология3_Профессиональное1
count,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,...,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000
mean,-0.193638,-0.997234,-0.998617,-0.996542,-0.997234,-0.329184,-0.290456,-0.186030,-0.023513,-0.479253,...,-0.964730,-0.982019,-0.994467,-0.997234,-0.999308,-0.998617,-0.991701,-0.999308,-0.999308,-0.999308
std,0.395285,0.074355,0.037178,0.069515,0.052541,0.470079,0.526139,0.389266,0.968429,0.502504,...,0.262014,0.132927,0.074200,0.052541,0.026298,0.037178,0.090750,0.026298,0.026298,0.026298
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
75%,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
max,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


####**x_1**
`df['МКБ' like]` + `df['ВредныеФакторы']` and OneHotEncoding

In [52]:
# # hardcode
mkb_list = df.filter(regex=re.compile('мкб', re.IGNORECASE), axis=1).columns.values.tolist()
print(mkb_list)
display(df[mkb_list].head())

['ПсихиатрияНаркология1_МКБ101', 'Офтальмология1_МКБ101', 'Оториноларингология1_МКБ101', 'ХирургНеИсп1_МКБ101', 'Гинекология1_МКБ101', 'Стоматология1_МКБ101', 'Дерматовенерология1_МКБ101', 'Рентгенология1_МКБ101', 'Неврология1_МКБ101', 'Терапия1_МКБ101', 'Хирургия1_МКБ101', 'Офтальмология1_МКБ102', 'Терапия2_МКБ101', 'Неврология2_МКБ101', 'ЛабораторныеИсследования1_МКБ101', 'ЭКГНеИсп1_МКБ101', 'УЗИДиагностика1_МКБ101', 'УЗИДиагностика2_МКБ101', 'ПсихиатрияНаркология2_МКБ101', 'Терапия1_МКБ102', 'ЛабораторныеИсследования2_МКБ101', 'Терапия1_МКБ103', 'Психиатрия1_МКБ101', 'Гинекология1_МКБ102', 'Рентгенология2_МКБ101', 'Гинекология2_МКБ101', 'Хирургия2_МКБ101', 'ПроцедурнаяМедсестра1_МКБ101', 'ФункциональнаяДиагностика1_МКБ101', 'УЗИДиагностика1_МКБ102', 'Стоматология2_МКБ101', 'Неврология1_МКБ102', 'Оториноларингология2_МКБ101', 'Дерматовенерология1_МКБ102', 'Дерматовенерология2_МКБ101', 'Дерматовенерология2_МКБ102', 'Хирургия1_МКБ102', 'Офтальмология2_МКБ101', 'Офтальмология2_МКБ102', 

,ПсихиатрияНаркология1_МКБ101,Офтальмология1_МКБ101,Оториноларингология1_МКБ101,ХирургНеИсп1_МКБ101,Гинекология1_МКБ101,Стоматология1_МКБ101,Дерматовенерология1_МКБ101,Рентгенология1_МКБ101,Неврология1_МКБ101,Терапия1_МКБ101,...,ЛабораторныеИсследования2_МКБ102,Гинекология3_МКБ101,ТерапевтНеИсп1_МКБ101,Эндоскопия1_МКБ101,ТерапевтНеИсп1_МКБ102,ХирургНеИсп1_МКБ102,Психиатрия1_МКБ102,Рентгенология1_МКБ102,Психиатрия3_МКБ101,Оториноларингология3_МКБ101
0,Z00.0,Z00.0,Z00.0,Z00.0,Z00.0,Z00.0,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,Z00.0,Z00.0,<NA>,<NA>,<NA>,<NA>,Z00.0,Z00.0,Z00.0,I11.9,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,Z00.0,H52.1,<NA>,<NA>,<NA>,<NA>,Z00.0,<NA>,<NA>,Z00.0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,<NA>,Z00.0,Z00.0,Z00.0,<NA>,Z00.0,<NA>,<NA>,Z00.0,Z00.0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,<NA>,H52.2,Z00.0,<NA>,<NA>,Z00.0,<NA>,<NA>,Z00.0,Z00.0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


Собираем в df_1 данные классификации МКБ и вредных факторов, затем взятые данные удаляем из df_string

In [53]:
mkb_harmful_list = []
mkb_harmful_list = df.filter(regex=re.compile('мкб', re.IGNORECASE), axis=1).columns.values.tolist()
mkb_harmful_list.append('ВредныеФакторы')
df_1 = df[mkb_harmful_list]
# df_1 = df_1.fillna('Empty')
df_string = df_string.drop(mkb_harmful_list, axis='columns')

In [54]:
# проверка заполненности
fills_count(df_1)

'Заполненность: ~13.08%'

In [55]:
df_1 = pd.get_dummies(df_1, columns=mkb_harmful_list)

In [56]:
x_1 = df_1.copy()

,ПсихиатрияНаркология1_МКБ101_F06.6,ПсихиатрияНаркология1_МКБ101_F06.7,ПсихиатрияНаркология1_МКБ101_F07.0,ПсихиатрияНаркология1_МКБ101_F41.0,ПсихиатрияНаркология1_МКБ101_F98.5,ПсихиатрияНаркология1_МКБ101_R25.1,ПсихиатрияНаркология1_МКБ101_Z00.0,Офтальмология1_МКБ101_E11.8,Офтальмология1_МКБ101_H01.0,Офтальмология1_МКБ101_H02.4,...,"ВредныеФакторы_6.2,18.1,23","ВредныеФакторы_6.2,4.1","ВредныеФакторы_6.2,4.1,6.2,4.1","ВредныеФакторы_6.2,9","ВредныеФакторы_8,11.1,11.4,18,18.2","ВредныеФакторы_8,11.1,18.2",ВредныеФакторы_9,"ВредныеФакторы_9,23","ВредныеФакторы_9,6.1,6.2","ВредныеФакторы_9,6.2"
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1961,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1962,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1972,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1973,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


####**WORK in progress** **x_2** and **y**



In [57]:
df_string.shape

(1446, 1216)

In [58]:
dict_string = display_nunique_cond(df_string)
dict_bow = {}
dict_token = {}
for k in dict_string:
    if len(dict_string[k]) <= 8:
        dict_token[k] = dict_string[k]
    else:
        dict_bow[k] = dict_string[k]
print(len(dict_string['ДиспансернаяГруппа']))
display(len(dict_bow))
display(len(dict_token))

Количество: 1216
5


99

1117

In [59]:
# j = 0
# for i in dict_token.keys():
#     print(f"LEN={len(dict_token[i])} KEY={i} VALUE={dict_token[i]}")
#     j += 1
#     if j >= 10: break

In [60]:
def categorize_reports(arg):
    y_categories = [
        ['Годен'],
        ['ГоденСКоррекциейЗрения', 'ГоденБезРаботНаВысотах', 'ОграниченноГоден'],
        ['НуждаетсяВДообследованииИЛечении', 'ВременноНегоден']
    ]
    item = arg.strip()
    if item in y_categories[0]:     return 'Годен'
    if item in y_categories[1]:     return 'ОграниченноГоден'
    if item in y_categories[2]:     return 'ВременноНегоден'

Колонки `ЗаключениеМКПредставление` и `Заключение` по сути дублируют колонку `ЗаключениеМК` только другими формулировками, поэтому удалим их

In [61]:
datareport_list = ['ЗаключениеМКПредставление', 'Заключение']
df_string = df_string.drop(datareport_list, axis='columns')

Объединим 6 вариантов в 3 с помощью функции `categorize_reports(arg)`

In [62]:
display(df_string['ЗаключениеМК'].value_counts())
df_string['ЗаключениеМК'] = [categorize_reports(arg) for arg in df_string['ЗаключениеМК']]
print()
display(df_string['ЗаключениеМК'].value_counts())

Годен                               1315
ГоденСКоррекциейЗрения               118
ВременноНегоден                        8
НуждаетсяВДообследованииИЛечении       2
ОграниченноГоден                       2
ГоденБезРаботНаВысотах                 1
Name: ЗаключениеМК, dtype: Int64

Годен               1315
ОграниченноГоден     121
ВременноНегоден       10
Name: ЗаключениеМК, dtype: int64

##Train, Validation data